# Methylation landscape analysis

In this notebook, I'll characterize the methylation landscape using output from [`BAT_summarize`](https://github.com/yaaminiv/killifish-hypoxia-RRBS/tree/main/output/05-analysis/summarize).

## 0. Prepare notebook for analysis

In [1]:
!pwd

/Users/yaaminivenkataraman/Documents/killifish-hypoxia-RRBS/code


In [2]:
cd ../output/05-analysis/

/Users/yaaminivenkataraman/Documents/killifish-hypoxia-RRBS/output/05-analysis


In [3]:
#!mkdir methylation-landscape

In [4]:
cd methylation-landscape/

/Users/yaaminivenkataraman/Documents/killifish-hypoxia-RRBS/output/05-analysis/methylation-landscape


In [5]:
!which bedtools

/opt/homebrew/bin/bedtools


In [14]:
bedtoolsDirectory = "/opt/homebrew/bin/"

## 1. Number of CpGs with 10-500x coverage

In [8]:
#Find files to concatenate
!find ../../04-calling/filtered/*sort.bedgraph

../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_20-N4_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_20-S1_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_20-S3_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_20-S4_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_5-N1_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_5-N2_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_5-S3_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_5-S4_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L2_OC-S1_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L3_20-N2_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L3_5-N3_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L3_5-S2_CG.sort.bedgraph
../../04-calling/filtered/190626_I114_FCH7TVNBBXY_L3_OC-N1_CG.sort.bedgraph
../../04-calling/f

In [16]:
#Create a union bedGraph
#Use N/A when there is no data for a CpG in a sample
#Define sample IDs
#Use sorted bedgraphs
#Cound the number of lines (CpGs) with data
!{bedtoolsDirectory}unionBedGraphs \
-filler N/A \
-i ../../04-calling/filtered/*sort.bedgraph \
| wc -l

 5403483


## 2. Global methylation

### 2a. Format data

In [83]:
#metilene output from all population comparison that includes all samples
!head ../summarize/all_pop/all_pop_metilene_N_S.txt
!wc -l ../summarize/all_pop/all_pop_metilene_N_S.txt

chr	pos	N_20-N4	N_5-N1	N_5-N2	N_20-N2	N_5-N3	N_20-N1	N_OC-N5	N_OC-N1	N_OC-N2	N_OC-N4	S_20-S1	S_20-S3	S_20-S4	S_5-S3	S_5-S4	S_5-S2	S_20-S2	S_5-S1	S_OC-S1	S_OC-S2	S_OC-S3	S_OC-S5
JXMV01056319.1	89	0.15	0.07	0.00	0.16	0.05	0.35	0.22	0.52	0.09	0.00	0.06	0.05	0.06	0.12	0.03	0.10	0.00	0.04	0.02	0.08	0.02	0.00
JXMV01056319.1	145	0.00	0.28	0.00	0.00	0.35	0.03	0.00	0.01	0.01	0.27	0.08	0.03	0.01	0.00	0.01	0.33	0.00	0.00	0.02	0.03	0.34	0.75
JXMV01057363.1	4077	0.01	0.00	0.04	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.17	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
JXMV01057363.1	4120	0.13	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00
JXMV01057451.1	2455	0.21	0.10	0.30	0.00	0.49	0.00	0.42	0.16	0.03	0.31	0.22	0.03	0.03	0.01	0.01	0.12	0.00	0.00	0.00	0.38	0.02	0.08
JXMV01058392.1	169	0.09	0.00	0.00	0.21	0.18	0.00	0.00	0.00	0.00	0.00	0.16	0.11	0.00	0.00	0.12	0.09	0.00	0.99	0.00	0.00	0.00	0.00
JXMV01058392.1	174	0.00	0.00	0.00	0.00	0.00	0.0

In [84]:
#Import data into pandas
#Check head
df = pd.read_table("../summarize/all_pop/all_pop_metilene_N_S.txt")
df.head(5)

,chr,pos,N_20-N4,N_5-N1,N_5-N2,N_20-N2,N_5-N3,N_20-N1,N_OC-N5,N_OC-N1,...,S_20-S4,S_5-S3,S_5-S4,S_5-S2,S_20-S2,S_5-S1,S_OC-S1,S_OC-S2,S_OC-S3,S_OC-S5
0,JXMV01056319.1,89,0.15,0.07,0.00,0.16,0.05,0.35,0.22,0.52,...,0.06,0.12,0.03,0.10,0.0,0.04,0.02,0.08,0.02,0.00
1,JXMV01056319.1,145,0.00,0.28,0.00,0.00,0.35,0.03,0.00,0.01,...,0.01,0.00,0.01,0.33,0.0,0.00,0.02,0.03,0.34,0.75
2,JXMV01057363.1,4077,0.01,0.00,0.04,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
3,JXMV01057363.1,4120,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
4,JXMV01057451.1,2455,0.21,0.10,0.30,0.00,0.49,0.00,0.42,0.16,...,0.03,0.01,0.01,0.12,0.0,0.00,0.00,0.38,0.02,0.08


In [85]:
#Average all samples for total genome methylation information and save as a new column
#NA are not included in averages
#Check output
df['average'] = df[['N_20-N4', 'N_5-N1', 'N_5-N2', 'N_20-N2', 'N_5-N3', 'N_20-N1', 'N_OC-N5', 'N_OC-N1', 'N_OC-N2', 'N_OC-N4', 'S_20-S1', 'S_20-S3', 'S_20-S4', 'S_5-S3', 'S_5-S4', 'S_5-S2', 'S_20-S2', 'S_5-S1', 'S_OC-S1', 'S_OC-S2', 'S_OC-S3', 'S_OC-S5']].mean(axis=1)
df.head(10)

,chr,pos,N_20-N4,N_5-N1,N_5-N2,N_20-N2,N_5-N3,N_20-N1,N_OC-N5,N_OC-N1,...,S_5-S3,S_5-S4,S_5-S2,S_20-S2,S_5-S1,S_OC-S1,S_OC-S2,S_OC-S3,S_OC-S5,average
0,JXMV01056319.1,89,0.15,0.07,0.00,0.16,0.05,0.35,0.22,0.52,...,0.12,0.03,0.10,0.0,0.04,0.02,0.08,0.02,0.00,0.099545
1,JXMV01056319.1,145,0.00,0.28,0.00,0.00,0.35,0.03,0.00,0.01,...,0.00,0.01,0.33,0.0,0.00,0.02,0.03,0.34,0.75,0.115909
2,JXMV01057363.1,4077,0.01,0.00,0.04,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.010000
3,JXMV01057363.1,4120,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.005909
4,JXMV01057451.1,2455,0.21,0.10,0.30,0.00,0.49,0.00,0.42,0.16,...,0.01,0.01,0.12,0.0,0.00,0.00,0.38,0.02,0.08,0.132727
5,JXMV01058392.1,169,0.09,0.00,0.00,0.21,0.18,0.00,0.00,0.00,...,0.00,0.12,0.09,0.0,0.99,0.00,0.00,0.00,0.00,0.088636
6,JXMV01058392.1,174,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.11,0.60,0.00,0.00,0.037273
7,JXMV01058392.1,183,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.015000
8,JXMV01058392.1,190,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,...,0.00,0.00,0.08,0.0,0.00,0.00,0.00,0.00,0.00,0.006364
9,JXMV01058392.1,197,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.05,0.0,0.00,0.13,0.00,0.00,0.00,0.008182


In [86]:
#Save dataframe in a tabular format and include N/As. Do not include quotes.
df.to_csv("all-samples-averages.bedgraph", sep = "\t", na_rep = "N/A", quoting = 3)

In [87]:
!head all-samples-averages.bedgraph
!wc -l all-samples-averages.bedgraph

	chr	pos	N_20-N4	N_5-N1	N_5-N2	N_20-N2	N_5-N3	N_20-N1	N_OC-N5	N_OC-N1	N_OC-N2	N_OC-N4	S_20-S1	S_20-S3	S_20-S4	S_5-S3	S_5-S4	S_5-S2	S_20-S2	S_5-S1	S_OC-S1	S_OC-S2	S_OC-S3	S_OC-S5	average
0	JXMV01056319.1	89	0.15	0.07	0.0	0.16	0.05	0.35	0.22	0.52	0.09	0.0	0.06	0.05	0.06	0.12	0.03	0.1	0.0	0.04	0.02	0.08	0.02	0.0	0.09954545454545456
1	JXMV01056319.1	145	0.0	0.28	0.0	0.0	0.35	0.03	0.0	0.01	0.01	0.27	0.08	0.03	0.01	0.0	0.01	0.33	0.0	0.0	0.02	0.03	0.34	0.75	0.11590909090909092
2	JXMV01057363.1	4077	0.01	0.0	0.04	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.17	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.010000000000000002
3	JXMV01057363.1	4120	0.13	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.005909090909090909
4	JXMV01057451.1	2455	0.21	0.1	0.3	0.0	0.49	0.0	0.42	0.16	0.03	0.31	0.22	0.03	0.03	0.01	0.01	0.12	0.0	0.0	0.0	0.38	0.02	0.08	0.1327272727272727
5	JXMV01058392.1	169	0.09	0.0	0.0	0.21	0.18	0.0	0.0	0.0	0.0	0.0	0.16	0.11	0.0	0.0	0.12	0.09	0.0	0.99	0.0	0.0	0.0	0.

### 2b. Number of methylated and unmethylated CpGs

In [88]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 all-samples-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 > .1) { print $26 }}' ${f} \
| wc -l

    5249


In [89]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 all-samples-averages.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 <= .1) { print $26 }}' ${f} \
| wc -l

    9646


In [98]:
#Calculate average methylation
! tail -n+2 all-samples-averages.bedgraph \
| awk '{ total += $26; count++ } END { print total/count }'

0.206832


## 3. Methylation by population

### 3a. New Bedford Harbor

In [90]:
#metilene output from all population comparison that includes all samples
!head ../summarize/all_pop/all_pop_mean_N.bedgraph
!wc -l ../summarize/all_pop/all_pop_mean_N.bedgraph

#chr	start	end	mean_N
JXMV01056319.1	88	89	0.161
JXMV01056319.1	144	145	0.095
JXMV01057363.1	4076	4077	0.005
JXMV01057363.1	4119	4120	0.013
JXMV01057451.1	2454	2455	0.202
JXMV01058392.1	168	169	0.048
JXMV01058392.1	173	174	0
JXMV01058392.1	182	183	0.022
JXMV01058392.1	189	190	0.006
   14896 ../summarize/all_pop/all_pop_mean_N.bedgraph


In [91]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/all_pop/all_pop_mean_N.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 > .1) { print $4 }}' ${f} \
| wc -l

    5291


In [94]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/all_pop/all_pop_mean_N.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 <= .1) { print $4 }}' ${f} \
| wc -l

    9604


In [99]:
#Calculate average methylation
! tail -n+2 ../summarize/all_pop/all_pop_mean_N.bedgraph \
| awk '{ total += $4; count++ } END { print total/count }'

0.206366


### 3b. Scorton Creek

In [93]:
#metilene output from all population comparison that includes all samples
!head ../summarize/all_pop/all_pop_mean_S.bedgraph
!wc -l ../summarize/all_pop/all_pop_mean_S.bedgraph

#chr	start	end	mean_S
JXMV01056319.1	88	89	0.0483333333333333
JXMV01056319.1	144	145	0.133333333333333
JXMV01057363.1	4076	4077	0.0141666666666667
JXMV01057363.1	4119	4120	0
JXMV01057451.1	2454	2455	0.075
JXMV01058392.1	168	169	0.1225
JXMV01058392.1	173	174	0.0683333333333333
JXMV01058392.1	182	183	0.00916666666666667
JXMV01058392.1	189	190	0.00666666666666667
   14896 ../summarize/all_pop/all_pop_mean_S.bedgraph


In [95]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/all_pop/all_pop_mean_S.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 > .1) { print $4 }}' ${f} \
| wc -l

    5249


In [96]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/all_pop/all_pop_mean_S.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 <= .1) { print $4 }}' ${f} \
| wc -l

    9646


In [100]:
#Calculate average methylation
! tail -n+2 ../summarize/all_pop/all_pop_mean_S.bedgraph \
| awk '{ total += $4; count++ } END { print total/count }'

0.207221


## 4. Methylation by population and oxygen treatment

### 4a. New Bedford Harbor

#### Hypoxia

In [101]:
#metilene output from all population comparison that includes all samples
!head ../summarize/20_5_N/N_mean_HY.bedgraph
!wc -l ../summarize/20_5_N/N_mean_HY.bedgraph

#chr	start	end	mean_HY
JXMV01051582.1	7434	7435	0.79
JXMV01051582.1	7456	7457	0.793333333333333
JXMV01051582.1	7487	7488	0.743333333333334
JXMV01051609.1	3477	3478	0.533333333333333
JXMV01051609.1	3494	3495	0.993333333333333
JXMV01051609.1	3501	3502	0.786666666666667
JXMV01051609.1	3511	3512	0.906666666666667
JXMV01051609.1	3517	3518	0.903333333333333
JXMV01051609.1	3522	3523	0.556666666666667
  176287 ../summarize/20_5_N/N_mean_HY.bedgraph


In [104]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_N/N_mean_HY.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 > .1) { print $4 }}' ${f} \
| wc -l

   61776


In [105]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_N/N_mean_HY.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 <= .1) { print $4 }}' ${f} \
| wc -l

  114510


In [106]:
#Calculate average methylation
! tail -n+2 ../summarize/20_5_N/N_mean_HY.bedgraph \
| awk '{ total += $4; count++ } END { print total/count }'

0.220691


#### Normoxia

In [107]:
#metilene output from all population comparison that includes all samples
!head ../summarize/20_5_N/N_mean_NO.bedgraph
!wc -l ../summarize/20_5_N/N_mean_NO.bedgraph

#chr	start	end	mean_NO
JXMV01051582.1	7434	7435	0.513333333333333
JXMV01051582.1	7456	7457	0.48
JXMV01051582.1	7487	7488	0.596666666666667
JXMV01051609.1	3477	3478	0.553333333333333
JXMV01051609.1	3494	3495	0.77
JXMV01051609.1	3501	3502	0.283333333333333
JXMV01051609.1	3511	3512	0.77
JXMV01051609.1	3517	3518	0.806666666666667
JXMV01051609.1	3522	3523	0.766666666666667
  176287 ../summarize/20_5_N/N_mean_NO.bedgraph


In [108]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_N/N_mean_NO.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 > .1) { print $4 }}' ${f} \
| wc -l

   58056


In [109]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_N/N_mean_NO.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 <= .1) { print $4 }}' ${f} \
| wc -l

  118230


In [110]:
#Calculate average methylation
! tail -n+2 ../summarize/20_5_N/N_mean_NO.bedgraph \
| awk '{ total += $4; count++ } END { print total/count }'

0.204056


### 4b. Scorton Creek

#### Hypoxia

In [112]:
#metilene output from all population comparison that includes all samples
!head ../summarize/20_5_S/S_mean_HY.bedgraph
!wc -l ../summarize/20_5_S/S_mean_HY.bedgraph

#chr	start	end	mean_HY
JXMV01052040.1	7873	7874	0.245
JXMV01052596.1	1404	1405	0.0025
JXMV01052596.1	16004	16005	0.0475
JXMV01052596.1	16037	16038	0.045
JXMV01052596.1	16083	16084	0.045
JXMV01052596.1	16087	16088	0.045
JXMV01052596.1	16094	16095	0
JXMV01054326.1	10897	10898	0.6475
JXMV01054326.1	10902	10903	0.835
   82306 ../summarize/20_5_S/S_mean_HY.bedgraph


In [113]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_S/S_mean_HY.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 > .1) { print $4 }}' ${f} \
| wc -l

   22694


In [114]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_S/S_mean_HY.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 <= .1) { print $4 }}' ${f} \
| wc -l

   59611


In [115]:
#Calculate average methylation
! tail -n+2 ../summarize/20_5_S/S_mean_HY.bedgraph \
| awk '{ total += $4; count++ } END { print total/count }'

0.164196


#### Normoxia

In [116]:
#metilene output from all population comparison that includes all samples
!head ../summarize/20_5_S/S_mean_NO.bedgraph
!wc -l ../summarize/20_5_S/S_mean_NO.bedgraph

#chr	start	end	mean_NO
JXMV01052040.1	7873	7874	0.2175
JXMV01052596.1	1404	1405	0
JXMV01052596.1	16004	16005	0
JXMV01052596.1	16037	16038	0
JXMV01052596.1	16083	16084	0
JXMV01052596.1	16087	16088	0
JXMV01052596.1	16094	16095	0
JXMV01054326.1	10897	10898	0.6875
JXMV01054326.1	10902	10903	0.7925
   82306 ../summarize/20_5_S/S_mean_NO.bedgraph


In [117]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_S/S_mean_NO.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 > .1) { print $4 }}' ${f} \
| wc -l

   22822


In [118]:
#Remove header
#Find CpGs with > 10% methylation
#Count number of CpGs
! tail -n+2 ../summarize/20_5_S/S_mean_NO.bedgraph \
| awk -F'\t' -v OFS='\t' '{if ($4 <= .1) { print $4 }}' ${f} \
| wc -l

   59483


In [119]:
#Calculate average methylation
! tail -n+2 ../summarize/20_5_S/S_mean_NO.bedgraph \
| awk '{ total += $4; count++ } END { print total/count }'

0.161531
